In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('vgsales.csv')

In [ ]:
def discretization(attr):
    inp = list(df[attr].to_numpy())
    midi = df[attr].describe()
    out = []
    for x in inp:
        if(x<=float(midi.loc['25%'])):
            out.append(attr+" sangat kecil")
        elif(x>float(midi.loc['25%']) and x<=float(midi.loc['50%'])):
            out.append(attr+" kecil")
        elif(x>float(midi.loc['50%']) and x<=float(midi.loc['75%'])):
            out.append(attr+" sedang")
        else:
            out.append(attr+" besar")
    return out
def multiDiscretization(attrs):
    for x in attrs:
        df[x+"_desc"] = pd.Series(discretization(x),index=df.index)
multiDiscretization(['NA_Sales','EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales'])

In [ ]:
df

In [ ]:
df.columns

In [ ]:
data = df[['Platform','Genre','Publisher']].astype(str).values.tolist()

In [ ]:
data = df[['Platform','Genre','Publisher','NA_Sales_desc','EU_Sales_desc', 'JP_Sales_desc', 'Other_Sales_desc', 'Global_Sales_desc']].astype(str).values.tolist()

In [ ]:
from sqlalchemy import create_engine
from pandas.io import sql
engine = create_engine('sqlite:///:memory:')
df.to_sql('vgsales',engine)

In [ ]:
df['Genre'].unique()

In [ ]:
type(data)

In [ ]:
#tidak perlu dikerjakan yg ini
def ARMformat(a,b,tb):
    collect = []
    for x in df[a].unique():
        q = "SELECT "+b+" FROM "+tb+" WHERE "+a+"='"+x+"'"
        res = pd.read_sql_query(q,engine)
        res=res[b].unique()
        resclean = []
        for i in res:
            if i!=None:
                resclean.append(i)
        #resmin = resclean[0:10] #terpaksa biar cepat saja
        resmin = resclean
        collect.append(list(resmin))
    return collect

In [ ]:
#tidak perlu dikerjakan yg ini
data = ARMformat('Platform','Publisher','vgsales')
len(data)

In [ ]:
#tidak perlu dikerjakan yg ini
i = 0
pre = df['Platform'].unique()
for dt in data:
    print(pre[i],dt,"\n")
    i+=1

FPGrowth

In [ ]:
!pip install pyfpgrowth

In [ ]:
import pyfpgrowth

In [ ]:
patterns = pyfpgrowth.find_frequent_patterns(data,200)
for p,d in patterns.items():
    if len(p)>5:
        print(p,":",d)

In [ ]:
rules = pyfpgrowth.generate_association_rules(patterns, 1.0)
for p,d in rules.items():
    """if len(p)>1:
        if d[0][0]=='PS2':
            print(p,"=>",d)"""
    if "JP_Sales" in d[0][0]:
        print(p,"=>",d)

Apriori

In [ ]:
!pip install efficient_apriori

In [ ]:
from efficient_apriori import apriori

In [ ]:
itemsets, rules = apriori(data, min_support=0.01,  min_confidence=1.0)

In [ ]:
hasil = []
for x in rules:
    hasil.append([str(x.lhs)+"=>"+str(x.rhs),x.support,x.confidence,x.lift,x.conviction])
    print(str(x.lhs)+"=>"+str(x.rhs),x.support,x.confidence,x.lift,x.conviction,sep="\t")

Tugas lakukan hal yang sama dengan menggunakan avocado sales
- lakukan discretization
- lakukan format data
- fp growth : panjang rule lebih dari 4, support: 0.05, confidence 1.0
- apriori : panjang rule lebih dari 4, support: 0.05, confidence 1.0